# 무신사 상품 리뷰 크롤링(crawling)

### 환경설정

In [1]:
#필요하면 설치하기
#!pip install webdriver-manager
#from selenium import webdriver
#from webdriver_manager.chrome import ChromeDriverManager
#driver = webdriver.Chrome(ChromeDriverManager().install())
#!pip install selenium

In [1]:
import warnings
warnings.filterwarnings("ignore")
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re
import time
import pandas as pd

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from html_table_parser import parser_functions as parser

 

In [3]:
#함수정의:검색어 조건에 따른 url생성
#무신사 스탠다드->후기순->벨트 및 양말 등 잡화제외 1위 상품
#def musinsa_searching(word):
#  url = "https://www.musinsa.com/app/goods/1149328" 
#  return url

In [10]:
#driver = webdriver.Chrome('chromedriver.exe')
#위치: C:\Users\rlawndms\boaz_삭제절대금지
#executable_path='C:/Users/rlawndms/boaz_삭제절대금지/chromedriver.exe'
#트레이닝/조거팬츠 후기순
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(options=options)
driver.get('https://www.musinsa.com/categories/item/001004?d_cat_cd=001004&brand=&list_kind=small&sort=emt_high&sub_sort=&page=1&display_cnt=90&group_sale=&exclusive_yn=&sale_goods=&timesale_yn=&ex_soldout=&kids=&color=&price1=&price2=&shoeSizeOption=&tags=&campaign_id=&includeKeywords=&measure=')
time.sleep(3)

In [11]:
 #user, gender,height,weight,item, size,content, evaluation(size_eval,bright_eval,color_eval,thick_eval)
user_list = []
gender_list =[]
height_list = []
weight_list =[]
item_list = []
size_list = []
star_list= []
content_list = []
size_eval_list =[]
bright_eval_list =[]
color_eval_list =[]
thick_eval_list =[]
#함수정의
def get_content(driver):
    #함수안에 html, soup 넣어놔야 페이지 넘어가서 바르게 긁어옴, 밖에 빼놓으면 첫페이지만 여러번 긁어진다.
    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')
    for i in range(10):
    
    #profile(gender,height,weight)
    #<p class="review-profile__body_information">남성, 177cm, 85kg</p>
        profile_before = soup.find_all('p','review-profile__body_information')
        profile_after = profile_before[i].text.split(',')
        try:
            gender = profile_after[0]
            height = profile_after[1]
            weight = profile_after[2]
        except:
            gender = ''
            height = ''
            weight = ''
#user :<p class="review-profile__name">LV 2 뉴비_95f88e16</p>           
#item: #/<a href="https://www.musinsa.com/app/goods/1231416/0" class="review-goods-information__name">테이퍼드 히든 밴딩 크롭 슬랙스 [더스티 베이지]</a>
#size :<span class="review-goods-information__option">
#content

        try:
            user = soup.find_all('p','review-profile__name')[i].text
            item = soup.find_all('a','review-goods-information__name')[i].text
            # '/n' 없애고 추출하기
            size = soup.find_all('span', 'review-goods-information__option')[i].text.strip().replace('\n','')
            content = soup.find_all('div','review-contents__text')[i].text
        except:
            user = ''
            item = ''
            size = ''
            content = ''
            
        #star
#->별 5개일때:<span class="review-list__rating__active" style="width: 100%"></span>
#->별 4개일때:<span class="review-list__rating__active" style="width: 80%"></span>
        stars = driver.find_elements_by_xpath('//*[@id="reviewListFragment"]/div['+str(i+1)+']/div[3]/span/span/span')
        try:
            for j in stars:
                a =j.get_attribute('style')
                if a[7:9]=='20':
                    star = 1
                elif a[7:9]=='40':
                    star = 2
                elif a[7:9]=='60':
                    star = 3
                elif a[7:9]=='80':
                    star = 4
                else:
                    star = 5
        except:
            star = ''
      

    #evaluation
        evaluation = soup.find_all('div', 'review-evaluation')
        try:
            size_eval = evaluation[i].find_all('span')[0].text
            bright_eval = evaluation[i].find_all('span')[1].text
            color_eval = evaluation[i].find_all('span')[2].text
            thick_eval = evaluation[i].find_all('span')[3].text
        except:
            size_eval = ''
            bright_eval = ''
            color_eval = ''
            thick_eval = ''

#user,gender,height,weight,item, size,content,star,size_eval,bright_eval,color_eval,thick_eval
        
        user_list.append(user)
        gender_list.append(gender)
        height_list.append(height)
        weight_list.append(weight)
        item_list.append(item)
        size_list.append(size)
        content_list.append(content)
        star_list.append(star)
        size_eval_list.append(size_eval)
        bright_eval_list.append(bright_eval)
        color_eval_list.append(color_eval)
        thick_eval_list.append(thick_eval)
  
   
        


In [12]:
#버튼 누르기 함수정의
def move_next(driver):    
    for i in range(4):
        get_content(driver)
        #페이지 2,3,4,5 넘어가기
        driver.find_element_by_css_selector('#reviewListFragment > div.nslist_bottom > div.pagination.textRight > div > a:nth-child(' + 
                                            str(int(4) + int(i)) + ')').send_keys(Keys.ENTER)
        time.sleep(2)
    get_content(driver)
    
#그다음 화살표'>'버튼누르기: (6,7,8...)있는 페이지로 넘어가기   
def move_arrow(driver):
    driver.find_element_by_css_selector('#reviewListFragment > div.nslist_bottom > div.pagination.textRight > div > a.fa.fa-angle-right.paging-btn.btn.next').send_keys(Keys.ENTER)
    

### 크롤링시작

In [13]:
#후기순 90개 자동으로 클릭하기
#뒤로가기 참고: https://m.blog.naver.com/mathesis_time/221970036063
# #searchList > li:nth-child(1) > div.li_inner > div.list_img > a > img
# #searchList > li:nth-child(2) > div.li_inner > div.list_img > a > img
# #searchList > li:nth-child(90) > div.li_inner > div.list_img > a > img
 #90개: 90
a = 1

for i in range(3):
    driver.find_element_by_css_selector('#searchList > li:nth-child(' +
                                            str(int(1) + int(i)) +
                                            ') > div.li_inner > div.list_img > a > img').click()
    time.sleep(2)
        
        #사이즈표
    try:
        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser')
        figure = soup.find('table',{'class':'table_th_grey'})
        p = parser.make2d(figure)
        figure_df = pd.DataFrame(data = p[1:],columns = p[0])
        figure_df.drop([0,1],inplace = True)
        print(figure_df)
    except:
        print("사이즈표 오류발생")
        
        #리뷰개수
    try:
        reviewNum = driver.find_element_by_xpath('//*[@id="estimate_style"]')
        reviewNum = reviewNum.text
        reviewNum = re.sub(r'[^0-9]','',reviewNum)
        reviewNum = int(reviewNum)
        print(reviewNum)
    except:
        print("리뷰개수 오류발생")
        

    user_list = []
    gender_list =[]
    height_list = []
    weight_list =[]
    item_list = []
    size_list = []
    star_list= []
    content_list = []
    size_eval_list =[]
    bright_eval_list =[]
    color_eval_list =[]
    thick_eval_list =[]
        
        #스타일후기리뷰 500개넘으면 진행,아니라면 진행 X
    if reviewNum >= 500:
        #크롤링
        #10:500개
        for i in range(1):
            try:
                move_next(driver)
                move_arrow(driver)
        #move_next(driver)
            except:
                time.sleep(2)
                    

                    
                    #뒤로가기
        driver.back()
        time.sleep(2)
    else:
        driver.back()
            
    time.sleep(2)    
    globals()["df"+str(a)] = pd.DataFrame({'user':user_list,
                                           'gender':gender_list,
                                           'height':height_list,
                                           'weight':weight_list,
                                            'item':item_list,
                                            'size':size_list,
                                            'star':star_list,
                                            'content':content_list,
                                            'size_eval':size_eval_list,
                                            'bright_eval':bright_eval_list,
                                            'color_eval':color_eval_list,
                                            'thick_eval':thick_eval_list})          
    #사이즈표와 리뷰 merge
    time.sleep(2) 
    globals()["merge_df"+str(a)] = pd.merge(globals()["df"+str(a)],figure_df,how = 'left',left_on = 'size',right_on = 'cm')
    a = a + 1            


    cm  총장 어깨너비 가슴단면 소매길이
2    M  70   65   61   59
3    L  72   67   64   61
4   XL  75   69   66   63
5  2XL  78   71   69   65
4105
   cm  총장 어깨너비 가슴단면 소매길이
2   S  68   62   57   54
3   M  70   63   59   55
4   L  72   64   61   56
5  XL  74   65   63   57
3300
   cm  총장 어깨너비  가슴단면 소매길이
2   S  69   60  57.5   57
3   M  71   62    60   59
4   L  73   64  62.5   61
5  XL  75   66    65   63
1833


In [17]:
merge_df3.head(5)
#merge_df2.head(5)
#merge_df3.head(5)

,user,gender,height,weight,item,size,star,content,size_eval,bright_eval,color_eval,thick_eval,cm,총장,어깨너비,가슴단면,소매길이
0,LV 4 뉴비_dbca74ca,여성,157cm,51kg,[패키지] NYC LOCATION HOOD T-SHIRTS (13COLOR) [LR...,민트/S ...,5,"커플 후드티로 구매했어요!가을에 입었던 거라 기모 없는 거 산 건데,구김 잘 가고 ...",보통이에요,밝아요,선명해요,얇아요,NaN,NaN,NaN,NaN,NaN
1,LV 3 안녕하세요정은,남성,178cm,71kg,[패키지] NYC LOCATION HOOD T-SHIRTS (13COLOR) [LR...,블랙/XL ...,5,기본 후드티로 가성비 가심비 다 챙겨서 넘 좋습니다. 기모 원단이 아니라서 두꺼운 ...,커요,보통이에요,보통이에요,보통이에요,NaN,NaN,NaN,NaN,NaN
2,LV 4 부루뚜마뚜,남성,174cm,90kg,[패키지] NYC LOCATION HOOD T-SHIRTS (13COLOR) [LR...,라이트네이비/XL ...,5,옷 형태도 이쁘게 제대로 나왔고 입었을때 느낌이 편안해요,커요,밝아요,선명해요,보통이에요,NaN,NaN,NaN,NaN,NaN
3,LV 4 뉴비_d42c3884,남성,182cm,76kg,[패키지] NYC LOCATION HOOD T-SHIRTS (13COLOR) [LR...,핑크/S ...,5,생각보다 사이즈 큽니다. 처음에 XL했다가 L로 했습니다. 다른 의류브랜드에서 보통...,커요,어두워요,보통이에요,보통이에요,NaN,NaN,NaN,NaN,NaN
4,LV 5 Ki Hyun,남성,178cm,76kg,[패키지] NYC LOCATION HOOD T-SHIRTS (13COLOR) [LR...,라이트 그레이/L ...,5,세미오버 스타일이라 핏 예쁘게 잡히네요저렴하게 후드 두개 잘샀습니다,보통이에요,보통이에요,보통이에요,보통이에요,NaN,NaN,NaN,NaN,NaN


In [7]:
#merge_df1.head()
merge_df1.head(3)

,user,gender,height,weight,item,size,star,content,size_eval,bright_eval,color_eval,thick_eval,cm,총장,허리단면,허벅지단면,밑위,밑단단면
0,LV 5 뉴비_79026c74,남성,171cm,64kg,Deep One Tuck Sweat Pants [Grey],M,5,많이 팔리는 이유가 있네요.막 입기 좋습니다. 만족합니다.,보통이에요,보통이에요,보통이에요,얇아요,M,103,32.5,34.5,34.5,24.5
1,LV 3 멤버_b7f404ee1c75,여성,167cm,55kg,Deep One Tuck Sweat Pants [Grey],S,5,너무너무 편해용.. 너무 편해서 손 많이 가고 무난해서 좋아용,커요,보통이에요,보통이에요,보통이에요,S,101,31.5,33,33,23.5
2,LV 6 비기기1,남성,177cm,68kg,Deep One Tuck Sweat Pants [Grey],L,5,사이즈 크게입어서 편하게 입기 좋아요 살짝 길긴하네요,보통이에요,보통이에요,보통이에요,보통이에요,L,105,34.5,36,36,25.5


In [9]:
merge_df2['size'].unique()

array(['M / Normal/기모', 'L / Normal/쭈리', 'L / Normal/기모',
       'XL / Normal/쭈리', 'M', 'XL / Normal/라이트 웨이트', 'S / Normal/쭈리',
       'Free / Normal/기모', 'L', 'XL / 라이트 웨이트', 'M / Normal/쭈리',
       'Free / Normal/쭈리', 'L / Low/여름용'], dtype=object)

In [68]:

merge_df1['size'].unique()

array(['★블랙(기모)★/L                                                                            , ★블랙(기모)★ L(+1000원)',
       '블랙/M                                                                            , 멜란지 M',
       '블랙/XL                                                                            , 오트밀 XL',
       '멜란지/M                                                                            , 화이트멜란지 M',
       '★블랙(기모)★/XL                                                                            , ★멜란지(기모)★ XL(+1000원)',
       '멜란지/XL                                                                            , 화이트멜란지 XL',
       '블랙/XL                                                                            , ★멜란지(기모)★ XL(+1000원)',
       '멜란지/L                                                                            , 멜란지 XL',
       '블랙/L                                                                            , 멜란지 L',
       '블랙/XL                                  

In [63]:
for i in range(3):
    globals()["ha"+str(int(1) + int(i))] = 5

# 비고

In [30]:
# //*[@id="estimate_style"]
# //*[@id="estimate_style"]
#reviewNum = driver.find_elements_by_xpath('//*[@id="estimate_style"]')
#for j in reviewNum:
#    b = j.get_attribute('estimate_style')
#b
#html = driver.page_source
#soup = BeautifulSoup(html,'html.parser')
#reviewNum = soup.find('span',{"class":"korSub"})
#print(reviewNum)
#print(reviewNum.text)
reviewNum = driver.find_element_by_xpath('//*[@id="estimate_style"]')
reviewNum = reviewNum.text
reviewNum = re.sub(r'[^0-9]','',reviewNum)
print(reviewNum)

20403


In [36]:
tabs = driver.window_handles
print(tabs)
#driver.switch_to_window(tabs[0])
#팝업창닫기 참고: https://ssanggo.tistory.com/81
#https://www.musinsa.com/app/goods/1551840,,, https://www.musinsa.com/app/goods/404949

['CDwindow-C838D6BC7FFF5FCE83CD5B2CAC2C91A0']


In [5]:
# 할인쿠폰 창 없애기->그냥 수동으로 했음

In [50]:
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

figure = soup.find('table',{'class':'table_th_grey'})
p = parser.make2d(figure)
figure_df = pd.DataFrame(data = p[1:],columns = p[0])
figure_df

,cm,총장,허리단면,허벅지단면,밑위,밑단단면
0,MY,가지고 계신 제품의 실측을 입력해 보세요~!,가지고 계신 제품의 실측을 입력해 보세요~!,가지고 계신 제품의 실측을 입력해 보세요~!,가지고 계신 제품의 실측을 입력해 보세요~!,가지고 계신 제품의 실측을 입력해 보세요~!
1,위 구매 내역의 사이즈를 저장하시겠습니까?,위 구매 내역의 사이즈를 저장하시겠습니까?,위 구매 내역의 사이즈를 저장하시겠습니까?,위 구매 내역의 사이즈를 저장하시겠습니까?,위 구매 내역의 사이즈를 저장하시겠습니까?,위 구매 내역의 사이즈를 저장하시겠습니까?
2,S,101,31.5,33,33,23.5
3,M,103,32.5,34.5,34.5,24.5
4,L,105,34.5,36,36,25.5
5,Free,110,34,34.5,35.5,25
6,S / 기모,101,31.5,33,33,23.5
7,M / 기모,103,32.5,34.5,34.5,24.5
8,L / 기모,105,34.5,36,36,25.5
9,Free / 기모,110,34,34.5,35.5,25


In [58]:
driver = webdriver.Chrome()
#해당 상품 링크_
#무신사->상의->후드티셔츠->무신사추천순->top1~5
#yale 후드티
driver.get('https://www.musinsa.com/app/goods/2314616')
time.sleep(3)

html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

figure = soup.find('table',{'class':'table_th_grey'})
p = parser.make2d(figure)
figure_df = pd.DataFrame(data = p[1:],columns = p[0])
figure.drop([0,1],inplace = True)
figure_df


,cm,총장,어깨너비,가슴단면,소매길이
0,MY,가지고 계신 제품의 실측을 입력해 보세요~!,가지고 계신 제품의 실측을 입력해 보세요~!,가지고 계신 제품의 실측을 입력해 보세요~!,가지고 계신 제품의 실측을 입력해 보세요~!
1,위 구매 내역의 사이즈를 저장하시겠습니까?,위 구매 내역의 사이즈를 저장하시겠습니까?,위 구매 내역의 사이즈를 저장하시겠습니까?,위 구매 내역의 사이즈를 저장하시겠습니까?,위 구매 내역의 사이즈를 저장하시겠습니까?
2,S,66,59,65.5,58
3,M,68,60.5,67.5,59
4,L,70,62,69.5,60
5,XL,72,63.5,71.5,61


In [61]:
#0,1행은 공통적으로 my사이즈와 저장하시겠습니까를 물어보는 행이라 삭제
figure_df.drop([0,1],axis =0,inplace = True)
figure_df

,cm,총장,어깨너비,가슴단면,소매길이
2,S,66,59,65.5,58
3,M,68,60.5,67.5,59
4,L,70,62,69.5,60
5,XL,72,63.5,71.5,61


In [29]:
# df생략없이 출력할 때
import numpy as np
np.set_printoptions(threshold=np.inf,linewidth=np.inf)
import pandas as pd
pd.set_option('display.max_columns',10)
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

In [ ]:
#팝업창없애기 original
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(options=options)
driver.get('https://www.musinsa.com/app/goods/2326935')
time.sleep(10)
try:
    #body > div > div > div > button > svg > path
    button = driver.find_element_by_css_selector('body > div > div > div > div > a.modal__btn.modal__btn--secondary')
    button.send_keys(Keys.ENTER)
    print("coupon close")
except:
    print("no button1")
time.sleep(5)

try:
    button2 = driver.find_element_by_css_selector('#divpop_goods_yale_8459 > form > button.btn.btn-close')
    button2.send_keys(Keys.ENTER)
except:
    print("no button2")
time.sleep(10)

* 참고
* 글로벌, 로컬: https://www.nemonein.xyz/2021/02/4908/